# 기업 주요 분석 과제 구현

## 0. Import

In [1]:
import os
from dotenv import load_dotenv

import pandas as pd

import plotly.graph_objects as go

from selenium import webdriver
from selenium.webdriver.common.by import By

In [2]:
load_dotenv()

True

## 1. 경쟁 기업 개요/현황

### 1-1. 기본 정보/규모

> 설립 연도, 위치, 조직 규모, 대표 이사, 주 사업 분야 등

- 잡코리아

In [111]:
class BaseInfoLoader:
    def __init__(self, url):
        self.url = url
        self.contents = []
    
    def load(self, in_pandas=False):
        # search url
        driver = webdriver.Chrome()
        driver.get(self.url)

        # find element with xpath
        elements = driver.find_elements(By.XPATH, '//*[@id="company-body"]/div[1]/div[1]/div/table')
        for element in elements:
            self.contents.append(element.text)
        
        # create base info dict
        keys = []
        values = []        
        for idx, text in enumerate(self.contents[0].split('\n')[1:]):
            keys.append(text) if idx%2==0 else values.append(text)
        self.info_dict = {
            key : value
            for key, value in zip(keys, values)
        }

        if in_pandas:
            info_df = pd.DataFrame([self.info_dict])
            return info_df
        else:
            return self.info_dict


In [112]:
url = "https://www.jobkorea.co.kr/Company/1696583"

loader = BaseInfoLoader(url)

In [ ]:
info_df = loader.load(in_pandas=True)

info_df

In [ ]:
loader.info_dict

{'산업': '이동전화기 제조업',
 '사원수': '129,095명 (2024.12.31)',
 '기업구분': '대기업',
 '설립일': '1969.12.01 (57년차)',
 '자본금': '8,975억 1천만원 (2024.12.31)',
 '매출액': '209조 5백억원 (2024.12.31)',
 '대표자': '전영현',
 '대졸초임': '5,163만원 상세보기',
 '주요사업': '휴대폰,컴퓨터,네트워크시스템,핵심칩,반도체부품,디스플레이패널,가전제품,의료기기,프린터 제조',
 '4대보험': '국민연금, 건강보험, 고용보험, 산재보험',
 '홈페이지': 'http://www.samsung.com/sec',
 '주소': '경기 수원시 영통구 삼성로 129 (매탄동, 삼성전자)',
 '계열사': '삼성그룹 스테코(주) 삼성전자판매(주) 삼성벤처투자(주) 삼성전자로지텍(주) (주)시큐아이 (주)씨브이네트 (주)멀티캠퍼스 삼성생명서비스손해사정(주) (주)미라콤아이앤씨 세메스(주) 에스디플렉스(주) 삼성중공업(주) 삼성SDI(주) 삼성전기(주) 삼성메디슨(주) (주)하만인터내셔널코리아 더보기'}

### 1-2. 비즈니스 모델


비즈니스 모델의 핵심 구성 요소:

- 고객 관계: 지속적인 관계를 조성하고 시간이 지남에 따라 고객 충성도를 구축하는 방법에 중점
- 가치 제안: 제품 또는 서비스를 독특하고 매력적이며 고객에게 가치 있는 것으로 만드는 요소를 정의
- 목표 고객: 이상적인 고객과 그들의 가장 시급한 니즈를 파악
- 수익 흐름: 비즈니스가 어떻게 수입을 창출하는지 설명
- 비용 구조: 비즈니스를 이끄는 주요 운영 비용을 강조
- 핵심 리소스: 성공에 중요한 물리적, 인적, 재정적 또는 지적 재산과 같은 필수 자산에 대해 설명
- 배포 채널: 제품 또는 서비스를 고객과 연결하고 고객에게 제공하는 방법을 설명

> 이후 분석 과제에서 '고객층', '차별화 요소' 에 대한 분석이 이루어지므로,

> '수익 흐름', '비용 구조' 를 중심으로 시각화 하는 것이 좋을 듯
>> 이를 위해서는 '재무제표'활용 필요

In [14]:
labels = [
    "DX부문 매출", "DS부문 매출", "SDC", "Harman",      # 1단계
    "총 매출", "매출총이익",      # 2단계(중간)
    "매출원가", "영업비용", "영업이익"       # 3단계(타겟)
]

# 단계별로 source와 target을 연결
source = [0, 1, 2, 3, 4, 4, 6, 6]
target = [4, 4, 4, 4, 5, 6, 7, 8]
values = [120, 50, 20, 10, 160, 40, 20, 20]

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15, thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
    ),
    link=dict(
        source=source,
        target=target,
        value=values
    )
))

fig.update_layout(title_text="삼성전자 수익구조", font_size=10)